---
title: "Stuff chain"
icon: "Book"
---

A chain that "stuffs" all the documents into the context to run summarization or extraction tasks.

In [1]:
from langchain.chains import create_stuff_documents_chain
from langchain.chat_models import init_chat_model
from langchain.documents import Document

In [2]:
documents = [
    Document(
        page_content=(
            "Richard Feynman was born on May 11, 1918, in Queens, New York. He showed an early "
            "interest in science, especially radios and engineering. As a teenager, he repaired "
            "radios as a hobby and even earned some money doing it.\n\n"
            "He attended the Massachusetts Institute of Technology (MIT) for his undergraduate "
            "studies and later earned his PhD in physics from Princeton University in 1942. At "
            "Princeton, he impressed many with his quick mind and problem-solving skills.\n\n"
            "After completing his PhD, he joined the Los Alamos Laboratory as part of the Manhattan Project."
        ),
        metadata={"source": "early_life"},
        id="1"
    ),
    Document(
        page_content=(
            "During World War II, Feynman worked on the Manhattan Project, the top-secret effort "
            "to build the first atomic bomb. He was based at Los Alamos Laboratory in New Mexico.\n\n"
            "There, he worked under physicist Hans Bethe and was known for his creativity and sense "
            "of humor. One of his habits was picking locks and cracking safes—not to steal secrets, "
            "but to prove how insecure they were.\n\n"
            "Feynman’s contributions helped the U.S. develop nuclear weapons, which were used in "
            "1945 to end the war."
        ),
        metadata={"source": "manhattan_project"},
        id="2"
    ),
    Document(
        page_content=(
            "After the war, Feynman became a professor at Cornell University and later at the "
            "California Institute of Technology (Caltech). In 1965, he won the Nobel Prize in "
            "Physics for his work on quantum electrodynamics, shared with Julian Schwinger and "
            "Sin-Itiro Tomonaga.\n\n"
            "He became famous for his lectures, especially the Feynman Lectures on Physics, which "
            "are still used today. In 1986, he served on the Rogers Commission that investigated "
            "the Space Shuttle Challenger disaster.\n\n"
            "Feynman died on February 15, 1988, in Los Angeles, California, after a long battle "
            "with cancer."
        ),
        metadata={"source": "later_career"},
        id="3"
    ),
]


## Summarization

In [3]:
model = init_chat_model("claude-opus-4-20250514", max_tokens=32_000)

summarizer = (
    create_stuff_documents_chain(
        model, 
        prompt="Summarize the contents into 3 key sentences."
    )
    .compile(name='summarize')
)

print(summarizer.invoke({"documents": documents})['result'])

Here are 3 key sentences summarizing Richard Feynman's life:

1. Richard Feynman was a brilliant physicist born in 1918 in Queens, New York, who showed early aptitude for science and engineering, eventually earning his PhD from Princeton in 1942.

2. During World War II, he made significant contributions to the Manhattan Project at Los Alamos, where he was known both for his scientific creativity and his playful habit of picking locks to demonstrate security flaws.

3. After the war, he became a renowned professor at Caltech, won the 1965 Nobel Prize in Physics for his work on quantum electrodynamics, created the famous Feynman Lectures on Physics, and served on the commission investigating the Challenger disaster before his death in 1988.


## Structured summary

In [4]:
from pydantic import BaseModel, Field
from typing import Optional, List

class BulletPoint(BaseModel):
    """Represents a single key idea with supporting document references."""
    content: str = Field(description="A concise bullet point summarizing a key idea.")
    source_doc_ids: List[str] = Field(default_factory=list, description="Document IDs supporting this bullet point.")

class StructuredSummary(BaseModel):
    """Structured summary including a title and supporting bullet points."""
    title: str = Field(description="A concise title summarizing the main theme.")
    bullet_points: List[BulletPoint] = Field(description="List of bullet points with supporting document references.")

model = init_chat_model("claude-opus-4-20250514", max_tokens=32_000)

chain = (
    create_stuff_documents_chain(
        model,
        response_format=StructuredSummary,
    )
    .compile(name='structured_summary_extractor')
)

structured_summary = chain.invoke({"documents": documents})['result']

print(f"Title: {structured_summary.title}")

for bullet_point in structured_summary.bullet_points:
    if not bullet_point.source_doc_ids:
        continue
    sources = ', '.join(bullet_point.source_doc_ids)
    print(f" * {bullet_point.content} ({sources})")


Title: Richard Feynman: Pioneering Physicist and Nobel Laureate
 * Born May 11, 1918, in Queens, New York, Feynman showed early aptitude for science and engineering, earning money repairing radios as a teenager (1)
 * Completed undergraduate studies at MIT and earned PhD in physics from Princeton University in 1942, impressing colleagues with his problem-solving abilities (1)
 * Worked on the Manhattan Project at Los Alamos Laboratory during WWII under Hans Bethe, contributing to the development of the atomic bomb (1, 2)
 * Known for his creativity and humor at Los Alamos, including his habit of picking locks and cracking safes to demonstrate security vulnerabilities (2)
 * Won the 1965 Nobel Prize in Physics for quantum electrodynamics work, shared with Julian Schwinger and Sin-Itiro Tomonaga (3)
 * Taught at Cornell University and Caltech, creating the influential Feynman Lectures on Physics that remain widely used today (3)
 * Served on the Rogers Commission investigating the 1986 S

## Extraction

In [5]:
from pydantic import BaseModel, Field
from typing import Optional, List

class Fact(BaseModel):
    """Fact about Richard Feynman."""
    content: str = Field(description="The content of the fact. No more than a few words.")
    iso_8601_date: Optional[str] = Field(default=None, description="The date associated with the fact if available. Formatted as YYYY-MM-DD.")
    source_doc_ids: List[str] = Field(default_factory=list, description="The document or documents in which the fact appeared.")

class Data(BaseModel):
    """Facts to extract"""
    facts: List[Fact]

model = init_chat_model("claude-opus-4-20250514", max_tokens=32_000)

chain = (
    create_stuff_documents_chain(
        model,
        response_format=Data,
    )
    .compile(name='fact_extractor')
)

fact_data = chain.invoke({"documents": documents})['result']

print("Extracted facts that have dates:")
for fact in fact_data.facts:
    if not fact.source_doc_ids:
        continue
    if not fact.iso_8601_date:
        continue
    date_str = f" on {fact.iso_8601_date}" if fact.iso_8601_date else ""
    sources = ', '.join(fact.source_doc_ids)
    print(f" * {date_str}: {fact.content} (sources: {sources})")


Extracted facts that have dates:
 *  on 1918-05-11: Born in Queens, New York (sources: 1)
 *  on 1942: PhD from Princeton University (sources: 1)
 *  on 1965: Won Nobel Prize in Physics (sources: 3)
 *  on 1986: Served on Rogers Commission (sources: 3)
 *  on 1988-02-15: Died in Los Angeles, California (sources: 3)
